# Advent of code 2023

Solutions are my own, if any external source including hints have been used it shall be mentioned and linked.


## Part1

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

    Five of a kind, where all five cards have the same label: AAAAA
    Four of a kind, where four cards have the same label and one card has a different label: AA8AA
    Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
    Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
    Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
    One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
    High card, where all cards' labels are distinct: 23456


In [7]:
from __future__ import annotations
from dataclasses import dataclass
from collections import Counter, defaultdict, deque

CARD_POWER_P1:dict[str,int] = {
    'A' : 13,
    'K' : 12,
    'Q' : 11,
    'J' : 10,
    'T': 9, 
    '9': 8,
    '8': 7,
    '7': 6,
    '6':5,
    '5': 4,
    '4': 3,
    '3': 2,
    '2': 1,
}

CARD_POWER_P2:dict[str,int] = {
    'A' : 13,
    'K' : 12,
    'Q' : 11,
    'T': 10, 
    '9': 9,
    '8': 8,
    '7': 7,
    '6':6,
    '5': 5,
    '4': 4,
    '3': 3,
    '2': 2,
    'J': 1
}

TYPES = {
    'five_oak':6,
    'four_oak':5,
    'full_house':4,
    'three_oak':3,
    'two_pair':2,
    'one_pair':1,
    'high_card':0,
}

TEST = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483"""

@dataclass
class Hand:
    cards: str
    bid: int
    joker_mode:bool=False

    @staticmethod
    def parse_hand(row:list[str], joker_mode:bool=False) -> Hand:
        hand, bid = row.split()
        return Hand(cards=hand.strip(), 
                    bid= int(bid),
                    joker_mode=joker_mode)

    def __post_init__(self):
        self.get_hand_type()
        self.get_hand_strength()

    def adjust_hand(self)->Counter:
        """changes original hand in the presence of
        joker as a wildcard"""
        hand = self.cards
        counter = Counter(hand)
        jokers = counter.get('J')
        if jokers and self.joker_mode:
            if jokers == 5:
                # corner case->hand with 5 jokers assume 5 aces
                self.cards = 'AAAAA'
                counter = Counter(self.cards)
            else:
                # find most common vals excluding Jokers
                top_values = [(elem, count) 
                            for elem, count in counter.most_common() 
                            if elem != 'J']
                # Extract all maximum values if there are ties
                max_count = top_values[0][1]
                # extract one of the most comon
                max_values = [elem 
                            for elem, count in counter.items() 
                            if count == max_count and elem != 'J']
                # replace 'J'
                replace_value  = sorted(max_values, 
                                        key=lambda count: CARD_POWER_P2[count[0]], 
                                        reverse=True)[0]
                replace_value = max_values[0] # either one works only type defined
                new_hand  = hand.replace("J", replace_value)
                # print(hand, new_hand, max_values)
                self.cards = new_hand
                
                counter = Counter(self.cards)
        self.old_cards = hand
        return counter
        
    def get_hand_type(self): #p2
        """Define the hand type"""
        counter = self.adjust_hand()
        values_counter = Counter(counter.values())
        if values_counter.get(5):
            self.type_ = 'five_oak' # of a kind
        elif values_counter.get(4):
            self.type_ = 'four_oak'
        elif values_counter.get(3):
            if values_counter.get(2):
                self.type_ = 'full_house'
            else:
                self.type_ = 'three_oak'
        elif values_counter.get(2):
            if values_counter[2] == 2:
                self.type_ = 'two_pair'
            else:
                self.type_ = 'one_pair'
        elif all(val ==1 for val in counter.values()):
            # print(values_counter, counter)
            self.type_ = 'high_card'
        else: 
            raise ValueError("hand type not recognised")

    def get_hand_strength(self):
        self.strength = TYPES[self.type_]


@dataclass
class Game:
    hands: list[Hand]
    joker_mode:bool=False

    @staticmethod
    def parse_game(puzzle:str, joker_mode:bool=False)-> Game:
        rows = puzzle.splitlines()
        return Game(hands= [Hand.parse_hand(row, joker_mode=joker_mode) 
                            for row in rows],
                    joker_mode=joker_mode)

    def __post_init__(self):
        self.get_sort_order()
        self.get_rank()

    def get_sort_order(self):
        """create hands list by type, sort each list by card power
        finally sort the type keys for weakest to strongest""" 
        
        CARD_POWER = CARD_POWER_P2 if self.joker_mode else CARD_POWER_P1
        # print(CARD_POWER)
        sort_order = defaultdict(list)
        for hand in self.hands:
            sort_order[hand.type_].append(hand)
        for _, hands in sort_order.items():
            hands.sort(key=lambda hand: [CARD_POWER[card] 
                                         for card in hand.old_cards]) # NOTE: sort by old card values!!! 
        # preparing for ranking lowest key (type) first
        self.sort_order = dict(sorted(sort_order.items(), 
                                      key= lambda k: TYPES[k[0]],
                                      reverse=False))

    def get_rank(self):
        sort_order = self.sort_order.copy()
        counter = 1
        for type_, hands in sort_order.items():
            for card in hands:
                card.rank = counter
                counter += 1

    def get_total_winnings(self):
        return sum(
            hand.rank * hand.bid
            for hand in self.hands
            )

In [8]:
game = Game.parse_game(puzzle=TEST)
assert game.get_total_winnings() == 6440
game_p2 = Game.parse_game(puzzle=TEST, joker_mode=True)
assert game_p2.get_total_winnings() == 5905

## Part 2

## Solutions

In [9]:
with open("puzzle_input/day07.txt") as file:
    puzzle = file.read()
game = Game.parse_game(puzzle=puzzle)
game_p2 = Game.parse_game(puzzle=puzzle, joker_mode=True)
print("part1", game.get_total_winnings())
print("part2", game_p2.get_total_winnings()) 

part1 254024898
part2 254115617
